## Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

## Load Data

In [2]:
data_path = '../IDMT-Traffic/datasets/df_main_encoded_only.csv'  
df = pd.read_csv(data_path)
df = df.drop(columns=['file'])
print(df.head(2))

   Unnamed: 0  is_background_encoded  date_time_encoded  location_encoded  \
0          18                      0                  0                 0   
1          19                      0                  0                 0   

  speed_kmh  daytime_encoded  weather_encoded  vehicle_encoded  \
0        30                1                0                1   
1        30                1                0                1   

   source_direction_encoded  microphone_encoded  ...  band_27_dB  band_28_dB  \
0                         1                   0  ...   37.024301   38.508511   
1                         1                   1  ...   34.516289   34.960402   

   band_29_dB  peak_dB_1  peak_freq_1  peak_dB_2  peak_freq_2  peak_dB_3  \
0   35.946349  50.180933    31.622777  49.528332  1000.000000  47.901831   
1   33.187933  54.903541  1000.000000  53.196406   794.328235  51.942355   

   peak_freq_3  octband_dB_mean  
0   794.328235        42.383307  
1   501.187234        42.050454

## Data Initialization & Split

In [3]:
target = 'daytime_encoded'  # Zielvariable
X = df.drop(columns=[target])  # Features (alle Spalten außer 'daytime')
y = df[target]  # Zielvariable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Reduzieren des Trainingsdatensatzes auf 200 Instanzen (Probedatensatz)
X_train = X_train[:200]
y_train = y_train[:200]

## Initialize Model

In [4]:
logreg = LogisticRegression(max_iter=1000, random_state=42)

## Train Model

In [5]:
# Trainieren des Modells
logreg.fit(X_train, y_train)

# Vorhersage auf Testdaten
y_pred = logreg.predict(X_test)

ValueError: could not convert string to float: 'UNK'